# Reasoning agents

We want to show how a reasoning model can plan, interact and iterate without any guidance to solve a problem.

Here we are using a reasoning model that also has the ability to use tools.

In [ ]:

import asyncio
import os
from openai import AsyncOpenAI

from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.text_plugin import TextPlugin
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.functions import KernelArguments, kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.kernel import Kernel
from dotenv import load_dotenv
from plugins import ChefPlugin
from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

load_dotenv()

service_id = "Expert"

if os.environ.get("AZURE_OPENAI_API_KEY"):
    print("Using Azure OpenAI")
    chat_completion_service = AzureChatCompletion(
        deployment_name=os.environ["AZURE_OPENAI_REASONING_DEPLOYMENT_NAME"],  
        api_key=os.environ["AZURE_OPENAI_API_KEY"],
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        service_id=service_id,
    )
else:
    print("Using GitHub Inference")
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings. 
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    base_url="https://models.inference.ai.azure.com"
    api_key=os.environ["GITHUB_TOKEN"]

    chat_completion_service = AzureAIInferenceChatCompletion(
        ai_model_id="o3-mini",
        api_key=api_key,
        endpoint=base_url,
        service_id=service_id,
    )

EXPERT_NAME = "Expert"

add the instances for the kernel

In [ ]:

kernel = Kernel()
kernel.add_service(chat_completion_service)
kernel.add_plugin(
    ChefPlugin(),
    plugin_name="Chef",
)

def _create_chat_completion_client() -> OpenAIChatCompletion:
    execution_settings = kernel.get_prompt_execution_settings_from_service_id(service_id=EXPERT_NAME)
    execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
    
    agent_expert_instance = ChatCompletionAgent(
        kernel=kernel,
        name=EXPERT_NAME,
        arguments=KernelArguments(settings=execution_settings),
        instructions="""
            Your sole responsiblity is provide the best recommendation possible using all tools and information available. Be mindful of the medical conditions and dietary restrictions of the user.
            If you found a recipe that matches the user's request, you can use the Chef plugin to provide the recipe details.
            DO NOT address the user.
            """,
    )

    return agent_expert_instance

run the agent

In [ ]:

agent_expert = _create_chat_completion_client()
thread: ChatHistoryAgentThread = None

is_complete: bool = False
while not is_complete:
    user_input = input("User:> ")
    if not user_input:
        continue

    if user_input.lower() == "exit":
        is_complete = True
        break

    if user_input.lower() == "reset":
        await thread.delete() if thread else None
        await thread.create() if thread else None
        print("[Conversation has been reset]")
        continue

    async for response in agent_expert.invoke(messages=user_input, thread=thread):
        print(f"# {response.role} - {response.name or '*'}: '{response.content}'")
        thread = response.thread